In [1]:
import pandas as pd

In [2]:
# Загрузка исходного XLSX файла
input_file = '/home/jupyter/datasphere/project/v2_ragas_npa_dataset_firstPart.xlsx'  # Укажите путь к вашему файлу
df = pd.read_excel(input_file)

In [3]:
df['question']

0      Каков объем экспорта услуг категории "Поездки"...
1      Каковы были основные мероприятия, реализуемые ...
2      Каков размер финансирования из федерального бю...
3      Какой комплексный центр социального обслуживан...
4      Какое значение имеет экологическое образование...
                             ...                        
495                Что входит в расходы госорганов Югры?
496    Как изменился объем переработки газа в 2009 году?
497    Что дает программа "Развитие материально-техни...
498            Какую долю занимают средства Нефтеюганск?
499    Субвенции ХМАО-Югры: как распределяются и что ...
Name: question, Length: 500, dtype: object

In [4]:
pattern = r"^\['(.*)'\]$"

df['contexts'] = df['contexts'].str.replace(pattern, r'\1', regex=True)


In [5]:
df['contexts'].head()

0    Увеличение объема экспорта услуг категории "По...
1    и | |развития | | |Ханты-Мансийского |исполнит...
2    не относящиеся к районам Крайнего Севера и при...
3    |2.2. |Центр социальной |240 к/мест |7580,0 |2...
4    (полного)общего образования и начального профе...
Name: contexts, dtype: object

In [6]:
answer = 'Оценка релевантности: набор фрагментов документов релевантен, так как в них содержатся ключевые слова и семантические значения, связанные с вопросом пользователя о реформах, связанных с ветеранами труда.\n\n**Ключевые слова:** ветераны труда, ветераны Великой Отечественной войны, ветераны боевых действий, труженики тыла, меры социальной поддержки, присвоение звания «ветеран труда», льготы, субсидии, законы «О ветеранах», «О мерах социальной поддержки отдельных категорий граждан».\n\nВ представленных фрагментах документов описывается законодательство и меры поддержки ветеранов труда и других категорий граждан, а также изменения и дополнения к этим законам. Это указывает на наличие информации о возможных реформах в этой области. Однако конкретные сведения о введении новых реформ или изменениях в законодательстве требуют более детального анализа и выделения конкретных цитат из фрагментов.'

In [7]:
%pip install ragas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [8]:
from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
from ragas import evaluate 

In [9]:
from datasets import Dataset
from ragas.metrics import faithfulness, answer_correctness

In [10]:
from ragas.llms import LangchainLLMWrapper

In [11]:
%pip install yandexcloud

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [12]:
from langchain_core.language_models import BaseLanguageModel
from langchain_core.embeddings import Embeddings
from langchain_community.chat_models import ChatYandexGPT
from langchain_community.llms import YandexGPT

In [13]:
llm = YandexGPT(iam_token='', folder_id='b1g5pkij4170jm1a1un6', temperature=0)

In [14]:
%pip install -U langchain_community tiktoken langchain-openai langchainhub langchain langgraph langchain_huggingface
%pip install -U pymilvus sentence-transformers fsspec s3fs yandexcloud transformers pip install langchain-experimental
%pip install --upgrade huggingface-hub
%pip install -qU  langchain_milvus
%pip install pydantic==2.6.0

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement install (from versions: none)

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
ERROR: No matching distribution found for install
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.4 which is incomp

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_milvus import Milvus
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoModel

model = AutoModel.from_pretrained("deepvk/USER-bge-m3", trust_remote_code=True) 

model_name = "deepvk/USER-bge-m3"
model_kwargs = {'device': 'cpu'}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
)


USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-09-12 17:30:55.675676: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-12 17:30:55.743591: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-12 17:30:56.780157: W tensorflow/compiler/tf2tensorrt/util

In [16]:
from ragas.run_config import RunConfig
from ragas.metrics.base import MetricWithLLM, MetricWithEmbeddings


# util function to init Ragas Metrics
def init_ragas_metrics(metrics, llm, embedding):
    for metric in metrics:
        if isinstance(metric, MetricWithLLM):
            metric.llm = llm
        if isinstance(metric, MetricWithEmbeddings):
            metric.embeddings = embedding
        run_config = RunConfig()
        metric.init(run_config)


In [17]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

# wrappers
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

init_ragas_metrics(
    metrics,
    llm=LangchainLLMWrapper(llm),
    embedding=LangchainEmbeddingsWrapper(embeddings),
)

NameError: name 'metrics' is not defined

In [ ]:
langchain_llm = LangchainLLMWrapper(langchain_llm=llm)

In [ ]:
# Вопрос от вашей RAG-системы
rag_question = 'Какова общая сумма по городу Когалыму за 2013-2014 годы?'

# Удаление лишних пробелов
df['question'] = df['question'].str.strip()
rag_question = rag_question.strip()

# Поиск вопроса в базе данных
matched_row = df[df['question'] == rag_question]

# Проверяем, найден ли вопрос
if not matched_row.empty:
    # Извлекаем правильный ответ и контекст
    ground_truth = matched_row['ground_truth'].values[0]
    contexts = matched_row['contexts'].values[0]
    # Ответ от вашей RAG-системы (замените на реальный ответ)
    rag_answer = "Оценка релевантности: набор фрагментов документов релевантен, так как в них содержатся ключевые слова и семантические значения, связанные с вопросом пользователя о реформах, связанных с ветеранами труда.\n\n**Ключевые слова:** ветераны труда, ветераны Великой Отечественной войны, ветераны боевых действий, труженики тыла, меры социальной поддержки, присвоение звания «ветеран труда», льготы, субсидии, законы «О ветеранах», «О мерах социальной поддержки отдельных категорий граждан».\n\nВ представленных фрагментах документов описывается законодательство и меры поддержки ветеранов труда и других категорий граждан, а также изменения и дополнения к этим законам. Это указывает на наличие информации о возможных реформах в этой области. Однако конкретные сведения о введении новых реформ или изменениях в законодательстве требуют более детального анализа и выделения конкретных цитат из фрагментов."

    # Подготовка данных для оценки
    data_samples = {
        'question': [rag_question],
        'answer': [rag_answer],
        'contexts': [[contexts]],
        'ground_truth': [ground_truth]
    }
    
    print(rag_question)
    print(contexts)
    print(ground_truth)
    print(rag_answer)
    
    # Создаем Dataset для оценки
    dataset = Dataset.from_dict(data_samples)

    # Оцениваем метрики
    score = evaluate(dataset, metrics=[answer_correctness], llm=llm, embeddings=embeddings)

    # Преобразуем результаты в pandas DataFrame для удобства
    df_score = score.to_pandas()
    print(df_score)

else:
    print("Вопрос не найден в базе данных.")

In [ ]:
dataset.to_pandas().head()

In [ ]:
matched_row['question']

In [ ]:
contexts = [contexts] if not isinstance(contexts, list) else contexts

In [ ]:
print(type(ground_truth))

In [ ]:
dataset